1. First Extract the text and images
 - Text -> chunk -> Embeddings
 - Images -> chunk -> Embeddings
In order to handle images and text, we will use the same model
 - Sclip model : provided by OpenAI, Model trained with images and text, it's open-source

2. Store the images in Base(64) format
3. Once the Embeddings created -> store them in vector store(FAISS, ChromaDB)  

Approach :
 - CLIP : Contrastive Language - Image Pre-Training
    - Vision Transformer + Transformer
 - Query - (CLIP EMBED) -> Retriever -> Context(Provide to LLM Prompt Template) -> LLM -> Output

#### Implementation

#### Step 1 : Pdf Extraction
- Till now for text, I used  PyPDFLoader
- Now, text + images extraction, PyMuPDF provides better performance

In [28]:
import fitz

from langchain_core.documents import Document
from transformers import CLIPProcessor, CLIPTextModel, CLIPModel
from PIL import Image


import torch
import numpy as np

from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage
from sklearn.metrics.pairwise import cosine_similarity


import os
import base64
import io

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


### LoadModel

In [20]:
# initialize the llm
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
api_key =os.getenv("claudeAPI")

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

- CLIP processor prepares the text or image for CLIP model. It handles tokenization, encoding and formatting

In [45]:
### Embedding functions
def embed_image(image_data):
    """Embed image using CLIP"""
    if isinstance(image_data, str):  # If path
        image = Image.open(image_data).convert("RGB")
    else:  # If PIL Image
        image = image_data
    
    inputs=clip_processor(images=image,return_tensors="pt")
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        # Normalize embeddings to unit vector
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()
    
def embed_text(text):
    """Embed text using CLIP."""
    inputs = clip_processor(
        text=text, 
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=77  # CLIP's max token length
    )
    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        # Normalize embeddings
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

- fitz.open(pdf_path) : extract the document

In [37]:
# Process pdf 
pdf_path = "/Users/akashdeepsangwan/Desktop/Code/GenAI/RAG/MultiModelLLM/multimodal_sample.pdf"
doc = fitz.open(pdf_path)

all_docs = []
all_embeddings = []
image_data_store = {} # store actual image data for LLM

# Text Splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, 
    chunk_overlap= 100,
    length_function = len
)


In [ ]:
doc # contains image_data and text_dat

Document('/Users/akashdeepsangwan/Desktop/Code/GenAI/RAG/MultiModelLLM/multimodal_sample.pdf')

In [46]:
for i, page in enumerate(doc) :
    # process text
    text = page.get_text()
    
    # remove all the empty space from the text
    if text.strip() :
        temp_doc = Document(page_content= text, metadata = {'page' : i, 'type' : "text"})
        text_chunks = splitter.split_documents([temp_doc])

    # Text chunking done -> create the embeddings of text
        for chunk in text_chunks :
            embeddings = embed_text(chunk.page_content)
            all_embeddings.append(embeddings)
            all_docs.append(chunk)


    """
    Process image :
    1) pdf image to PIL Format
    2) store it in base 64
    3) Create the emebeddings
    
    
    """

    for img_index, img in enumerate(page.get_images(full=True)):
        try:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            
            # Convert to PIL Image
            pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
            
            # Create unique identifier
            image_id = f"page_{i}_img_{img_index}"
            
            # Store image as base64 for later use with GPT-4V
            buffered = io.BytesIO()
            pil_image.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode()
            image_data_store[image_id] = img_base64
            
            # Embed image using CLIP
            embedding = embed_image(pil_image)
            all_embeddings.append(embedding)
            
            # Create document for image
            image_doc = Document(
                page_content=f"[Image: {image_id}]",
                metadata={"page": i, "type": "image", "image_id": image_id}
            )
            all_docs.append(image_doc)
            
        except Exception as e:
            print(f"Error processing image {img_index} on page {i}: {e}")
            continue

doc.close()


In [ ]:
# pdf processed -> Image and text_extracted -> Embeddings created

In [ ]:
all_docs 

[Document(metadata={'page': 0, 'type': 'text'}, page_content='Annual Revenue Overview\nThis document summarizes the revenue trends across Q1, Q2, and Q3. As illustrated in the chart\nbelow, revenue grew steadily with the highest growth recorded in Q3.\nQ1 showed a moderate increase in revenue as new product lines were introduced. Q2 outperformed\nQ1 due to marketing campaigns. Q3 had exponential growth due to global expansion.'),
 Document(metadata={'page': 0, 'type': 'image', 'image_id': 'page_0_img_0'}, page_content='[Image: page_0_img_0]')]

In [56]:
# Create Unified FAISS Vector-store
embeddings_array = np.array(all_embeddings)

vector_store = FAISS.from_embeddings(
    text_embeddings = [(doc.page_content, emb) for doc,emb in zip(all_docs, embeddings_array)],
    embedding = None, # we are using precomputed embeddings
    metadatas = [doc.metadata for doc in all_docs]
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [53]:
llm = init_chat_model(
    "claude-haiku-4-5-20251001",
    temperature = 0.7,
    api_key = os.getenv("claudeAPI")
)

In [63]:
# retrieval model
def retrieve_multimodel(query, k=5) :
    query_embedding = embed_text(query) # embeddings function

    #Search in Unified vector store
    results = vector_store.similarity_search_by_vector(
        embedding = query_embedding,
        k=k
    )
    return results

In [65]:
def create_multimodal_message(query, retrieved_docs):
    """Create a message with both text and images"""
    content = []
    
    # Add the query
    content.append({
        "type": "text",
        "text": f"Question: {query}\n\nContext:\n"
    })
    
    # Separate text and image documents
    text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
    image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]
    
    # Add text context
    if text_docs:
        text_context = "\n\n".join([
            f"[Page {doc.metadata['page']}]: {doc.page_content}"
            for doc in text_docs
        ])
        content.append({
            "type": "text",
            "text": f"Text excerpts:\n{text_context}\n"
        })
    
    # Add images
    for doc in image_docs:
        image_id = doc.metadata.get("image_id")
        if image_id and image_id in image_data_store:
            content.append({
                "type": "text",
                "text": f"\n[Image from page {doc.metadata['page']}]:\n"
            })
            content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_data_store[image_id]}"
                }
            })
    
    # Add instruction
    content.append({
        "type": "text",
        "text": "\n\nPlease answer the question based on the provided text and images."
    })
    
    return HumanMessage(content=content)

In [66]:
def multimodal_pdf_rag_pipeline(query):
    """Main pipeline for multimodal RAG."""
    # Retrieve relevant documents
    context_docs = retrieve_multimodel(query, k=5)
    
    # Create multimodal message
    message = create_multimodal_message(query, context_docs)
    
    # Get response from GPT-4V
    response = llm.invoke([message])
    
    # Print retrieved context info
    print(f"\nRetrieved {len(context_docs)} documents:")
    for doc in context_docs:
        doc_type = doc.metadata.get("type", "unknown")
        page = doc.metadata.get("page", "?")
        if doc_type == "text":
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"  - Text from page {page}: {preview}")
        else:
            print(f"  - Image from page {page}")
    print("\n")
    
    return response.content

In [ ]:
if __name__ == "__main__":
    # Example queries
    queries = [
        "What does the chart on page 1 show about revenue trends?",
        "Summarize the main findings from the document",
        "What visual elements are present in the document?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)
        answer = multimodal_pdf_rag_pipeline(query)
        print(f"Answer: {answer}")
        print("=" * 70)